<a href="https://colab.research.google.com/github/Calebmonroe/DS2/blob/main/HW6_Caleb_Monroe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Homework #6 - Support Vector Machine Classifiers and Prediction Error Analysis

In this assignment, we will explore the power of support vector machine classifiers applied to several multivariate datasets.


# Package Imports

##Imports and Function Definitions

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import sklearn.metrics as metrics
import seaborn as sns
import cv2
sns.axes_style("whitegrid")
sns.set_context("paper")
from keras.datasets import mnist
from sklearn import utils
from sklearn import preprocessing
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import add_dummy_feature
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import SGDRegressor
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline

from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import mean_squared_error
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import precision_recall_fscore_support
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

from sklearn.svm import LinearSVC
from sklearn.svm import SVC

#np.random.seed(42)

In [2]:
#import sys
#np.set_printoptions(threshold=sys.maxsize)

In [3]:
def regression_results(y_true, y_pred):

    # Regression metrics
    explained_variance=metrics.explained_variance_score(y_true, y_pred)
    mean_absolute_error=metrics.mean_absolute_error(y_true, y_pred)
    mse=metrics.mean_squared_error(y_true, y_pred)
#    mean_squared_log_error=metrics.mean_squared_log_error(y_true, y_pred)
    median_absolute_error=metrics.median_absolute_error(y_true, y_pred)
    r2=metrics.r2_score(y_true, y_pred)

    print('explained_variance: ', round(explained_variance,4))
    print('mean_absolute_error: ', round(mean_absolute_error,4))
#    print('mean_squared_log_error: ', round(mean_squared_log_error,4))
    print('r2: ', round(r2,4))
    print('MAE: ', round(mean_absolute_error,4))
    print('MSE: ', round(mse,4))
    print('RMSE: ', round(np.sqrt(mse),4))


In [4]:
def model_acc_t(res):

    m1 = res['pred']; m2 = res['actual']
    sum12 = sum(m1==m2)
    pct=sum12/len(res)*100

    print("Your model made ", pct, "percent correct predictions\n")
    return
#X_train[0]

In [5]:
#model_acc2(res_df,type) # 0 for setosa, 1 for versi, 2 for virg

# need condition for each type
def model_acc2(res,type):

    if (type==0):
      m1 = res['pred']>=0.5
      m2 = res['actual']==1
    elif (type==1):
      m1 = res['pred']>=0.5
      m2 = res['actual']==1
    elif (type==2):
      m1 = res['pred']>=0.5
      m2 = res['actual']==1


#    m1 = res['pred']>=0.5
#    m2 = res['actual']==1
    sum12=sum(m1==m2)
    pct=sum12/len(res)*100
    print("Your model made ", pct, "percent correct predictions\n")
    return

In [6]:
def model_parm_std_err(xdata, ydata, model): # assumes xtest and ytest are not dfs
# this function was written for univariate regression,
# not yet tested for multivariate

# if you just want to check whether this algorithm works, use statsmodels

# using this with scaled data (both x and y) gives the same slope
# but a different intercept

    df_xt = pd.DataFrame(xdata)
    df_yt = pd.DataFrame(ydata)

    modelf = model

    N = len(df_xt)
    p = len(df_xt.columns) + 1  # plus one because LinearRegression adds an intercept term

    X_with_intercept = np.empty(shape=(N, p), dtype=float)
    X_with_intercept[:, 0] = 1
    X_with_intercept[:, 1:p] = xdata


    beta_hat = np.linalg.inv(X_with_intercept.T @ X_with_intercept) @ X_with_intercept.T @ ydata

    #print(beta_hat)

    modelf.fit(xdata,ydata)

    slope = modelf.coef_[0][0]
    intercept = modelf.intercept_[0]

    y_hat = modelf.predict(df_xt)
    residuals = ydata - y_hat

    residual_sum_of_squares = residuals.T @ residuals
    sigma_squared_hat = residual_sum_of_squares[0, 0] / (N - p)
    var_beta_hat = np.linalg.inv(X_with_intercept.T @ X_with_intercept) * sigma_squared_hat


# var_beta hat output errors on intercept first, then slope1, slope2, etc.
    for p_ in range(p):
      standard_error = var_beta_hat[p_, p_] ** 0.5
#      print(var_beta_hat.shape)
#      print(f"SE(beta_hat[{p_}]): {standard_error}")

    print ("slope:\t", slope, "+/-", var_beta_hat[1,1]**0.5, "\nintercept:\t", intercept, "+/-", var_beta_hat[0,0]**0.5)


In [7]:
def lasso_model_parm_std_err(xdata, ydata, model): # assumes xtest and ytest are not dfs
# this function was written for univariate regression for the lasso model which
# is formatted differently

# if you just want to check whether this algorithm works, use statsmodels

# using this with scaled data (both x and y) gives the same slope
# but a different intercept

    df_xt = pd.DataFrame(xdata)
    df_yt = pd.DataFrame(ydata)

    modelf = model

    N = len(df_xt)
    p = len(df_xt.columns) + 1  # plus one because LinearRegression adds an intercept term

    X_with_intercept = np.empty(shape=(N, p), dtype=float)
    X_with_intercept[:, 0] = 1
    X_with_intercept[:, 1:p] = xdata


    beta_hat = np.linalg.inv(X_with_intercept.T @ X_with_intercept) @ X_with_intercept.T @ ydata

    #print(beta_hat)

    modelf.fit(xdata,ydata)

# format for lasso
    slope = modelf.coef_[0]
    intercept = modelf.intercept_[0]


    y_hat = modelf.predict(df_xt)
    residuals = ydata - y_hat

    residual_sum_of_squares = residuals.T @ residuals
    sigma_squared_hat = residual_sum_of_squares[0, 0] / (N - p)
    var_beta_hat = np.linalg.inv(X_with_intercept.T @ X_with_intercept) * sigma_squared_hat


# var_beta hat output errors on intercept first, then slope1, slope2, etc.
    for p_ in range(p):
      standard_error = var_beta_hat[p_, p_] ** 0.5
#      print(var_beta_hat.shape)
#      print(f"SE(beta_hat[{p_}]): {standard_error}")

    print ("slope:\t", slope, "+/-", var_beta_hat[1,1]**0.5, "\nintercept:\t", intercept, "+/-", var_beta_hat[0,0]**0.5)

In [8]:
def mv_model_parm_std_err(xdata, ydata, model): # assumes xdata and ydata are not dfs
# this function was written for multivariate regression for the lasso model which
# is formatted differently

    df_xt = pd.DataFrame(xdata)
    df_yt = pd.DataFrame(ydata)

    modelf = model

    N = len(df_xt)
    p = len(df_xt.columns) + 1  # plus one because LinearRegression adds an intercept term

    X_with_intercept = np.empty(shape=(N, p), dtype=float)
    X_with_intercept[:, 0] = 1
    X_with_intercept[:, 1:p] = xdata

    beta_hat = np.linalg.inv(X_with_intercept.T @ X_with_intercept) @ X_with_intercept.T @ ydata

    #print(beta_hat)

    modelf.fit(xdata,ydata)

# format for lasso
    slope = []
    for i in range(len(modelf.coef_)):
      slope.append(modelf.coef_[i])

#    slope = modelf.coef_[0]
    intercept = modelf.intercept_[0]


    y_hat = modelf.predict(df_xt)
    residuals = ydata - y_hat

    residual_sum_of_squares = residuals.T @ residuals
    sigma_squared_hat = residual_sum_of_squares[0, 0] / (N - p)
    var_beta_hat = np.linalg.inv(X_with_intercept.T @ X_with_intercept) * sigma_squared_hat


# var_beta hat output errors on intercept first, then slope1, slope2, etc.
    for p_ in range(p):
      standard_error = var_beta_hat[p_, p_] ** 0.5
#      print(var_beta_hat.shape)
#      print(f"SE(beta_hat[{p_}]): {standard_error}")

    for p_ in range(p):
      std_err = var_beta_hat[p_,p_]**0.5
      if(p_==0):
        print("intercept:\t",intercept,"+/-",std_err,"\n")
        continue
#          print("slope",p_,":\t",slope[p_],"+/-",std_err,"\n")
#      else:
      print("slope",p_,":\t",slope[p_-1],"+/-",std_err,"\n")

#    print ("slope:\t", slope, "+/-", var_beta_hat[1,1]**0.5, "\nintercept:\t", intercept, "+/-", var_beta_hat[0,0]**0.5)

As we did in previous chapters, let's define the default font sizes to make the figures prettier:

In [9]:
import matplotlib.pyplot as plt

plt.rc('font', size=14)
plt.rc('axes', labelsize=14, titlesize=14)
plt.rc('legend', fontsize=14)
plt.rc('xtick', labelsize=10)
plt.rc('ytick', labelsize=10)

# Exercise 1 - SGD and SVM classifier on Obesity Data

This dataset is from Kaggle and looks at personal demographics, along with eating and exercise factors, as they are related to what degree a person is considered obese.  

https://www.kaggle.com/datasets/aravindpcoder/obesity-or-cvd-risk-classifyregressorcluster


Using the make_pipeline method with StandardScaler(),
train 1) a multivariate SGD classifier and 2) an SVM classifier in order to predict the level of obesity a person has from various eating and exercise habits.

Next, add Polynomial Features to your LinearSVC pipeline, then retrain, reevaluate, and compare your resulting model's performance to the previous SGD classifier and the SVM without polynomial features.   


Be sure to:
* evaluate each model's accuracy on the training data using both the **score** method and using **cross_val_score**
* present each model's precision, recall, and f1 scores with respect to the training set

* evaluate each model's accuracy on the test set (using your model to make test predictions, then comparing those test predictions against the actual target test data)
* present each model's precision, recall, and f1 scores with respect to the training set

* present your final discussion/comparison in a seprate Markdown cell at the end of your analysis

## Preparing the dataframe

This dataset was retrieve from Kaggle.  It contains peronal demographic information and attributes related to eating habits for 2111 individuals.  The question is, **how well can we predict obesity based on calorie intake and expenditure**

Personal Demographic features are:
Gender, Age, Height, Weight, family history

Attributes related to eating habits:
* FAVC - Frequent of consumption of high caloric food (yes/no)
* FCVC - Frequency of consumption of vegetables (servg per day)
* NCP - Number of main meals (1-4)
* CAEC - Consumption of food between meals (Sometimes', 'Frequently', 'Always', 'no')
* SMOKE - Smoking (yes/no)
* CH20 - Daily consumption of water (1-3 L)
* SCC - Caloric consumption monitoring (yes/no)
* FAF - Physical activity frequency (0-3 hours)
* TUE - Time using technology devices (0-2 hours)
* CALC - Consumption of alcohol ('no', 'Sometimes', 'Frequently', 'Always')
* MTRANS - Transportation used ('Public_Transportation', 'Walking', 'Automobile', 'Motorbike', 'Bike')
* NObeyesdad - Degree of obesity (Underweight, Normal, Overweight_I,
 Overweight_II, Obesity_I, Obesity_II, Obesity_III)

In [10]:
#df = pd.read_csv('ObesityDataSet.csv')

web_path = 'http://public.gettysburg.edu/~rjohnson/ds325/' #if using data over web
df = pd.read_csv(web_path+'ObesityDataSet.csv')

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2111 entries, 0 to 2110
Data columns (total 17 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Gender                          2111 non-null   object 
 1   Age                             2111 non-null   float64
 2   Height                          2111 non-null   float64
 3   Weight                          2111 non-null   float64
 4   family_history_with_overweight  2111 non-null   object 
 5   FAVC                            2111 non-null   object 
 6   FCVC                            2111 non-null   float64
 7   NCP                             2111 non-null   float64
 8   CAEC                            2111 non-null   object 
 9   SMOKE                           2111 non-null   object 
 10  CH2O                            2111 non-null   float64
 11  SCC                             2111 non-null   object 
 12  FAF                             21

Our task will be to train a SVM classifier that can predict a person's degree of obesity based on their eating and exercise habits.  Since we want a person's obesity class to be a function only of these variables, we will intentionally ignore Age, Height, and Weight (which most certainly are correlated with obesity).  The strategy will be to:

* convert categorical features to numerical
* examine variable distribution for stratifying purposes
* create pipeline including standard scaler
* train svm classifier


In [12]:
# rename columns
df.rename(columns={"family_history_with_overweight": "hist"}, inplace=True)
df.rename(columns={"NObeyesdad": "obes"}, inplace=True)

In [13]:
# I'm manually encoding the obesity classification because when I used
# LabelEncoder(), it put them in alphabetical order, which wasn't helpful

df['obes'].replace({'Insufficient_Weight':0, 'Normal_Weight':1,
                                'Overweight_Level_I':2, 'Overweight_Level_II':3,
                                'Obesity_Type_I':4, 'Obesity_Type_II':5,
                                'Obesity_Type_III':6},inplace=True)

In [14]:
# here we encode all of the non-numerical features
# using LabelEncoder and replace our dataframe
le_hist = LabelEncoder()
dft = le_hist.fit_transform(df['hist'])
df['hist'] = dft

le_gen = LabelEncoder()
dft = le_gen.fit_transform(df['Gender'])
df['Gender'] = dft

le_favc = LabelEncoder()
dft = le_favc.fit_transform(df['FAVC'])
df['FAVC'] = dft

le_caec = LabelEncoder()
dft = le_caec.fit_transform(df['CAEC'])
df['CAEC'] = dft

le_smoke = LabelEncoder()
dft = le_smoke.fit_transform(df['SMOKE'])
df['SMOKE'] = dft

le_scc = LabelEncoder()
dft = le_scc.fit_transform(df['SCC'])
df['SCC'] = dft

le_calc = LabelEncoder()
dft = le_calc.fit_transform(df['CALC'])
df['CALC'] = dft

le_mtrans = LabelEncoder()
dft = le_mtrans.fit_transform(df['MTRANS'])
df['MTRANS'] = dft

In [15]:
df

,Gender,Age,Height,Weight,hist,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,obes
0,0,21.000000,1.620000,64.000000,1,0,2.0,3.0,2,0,2.000000,0,0.000000,1.000000,3,3,1
1,0,21.000000,1.520000,56.000000,1,0,3.0,3.0,2,1,3.000000,1,3.000000,0.000000,2,3,1
2,1,23.000000,1.800000,77.000000,1,0,2.0,3.0,2,0,2.000000,0,2.000000,1.000000,1,3,1
3,1,27.000000,1.800000,87.000000,0,0,3.0,3.0,2,0,2.000000,0,2.000000,0.000000,1,4,2
4,1,22.000000,1.780000,89.800000,0,0,2.0,1.0,2,0,2.000000,0,0.000000,0.000000,2,3,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2106,0,20.976842,1.710730,131.408528,1,1,3.0,3.0,2,0,1.728139,0,1.676269,0.906247,2,3,6
2107,0,21.982942,1.748584,133.742943,1,1,3.0,3.0,2,0,2.005130,0,1.341390,0.599270,2,3,6
2108,0,22.524036,1.752206,133.689352,1,1,3.0,3.0,2,0,2.054193,0,1.414209,0.646288,2,3,6
2109,0,24.361936,1.739450,133.346641,1,1,3.0,3.0,2,0,2.852339,0,1.139107,0.586035,2,3,6


# Solution

In [16]:
X = df[['hist','Gender','FAVC', 'CAEC', 'SMOKE', 'SCC', 'CALC', 'MTRANS', 'FCVC', 'NCP', 'CH2O', 'FAF', 'TUE']]
y = df['obes'] #defining X and y variables for train test split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42) #train test split

In [17]:
sgd_pipeline = make_pipeline(StandardScaler(), SGDClassifier(max_iter=1000, tol=1e-3))#creating SGD pipeline
sgd_pipeline.fit(X_train, y_train)#fitting new model

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('sgdclassifier', SGDClassifier())])

In [18]:
from sklearn.metrics import accuracy_score #importing  accuracy score
sgd_train_accuracy = accuracy_score(y_train, sgd_pipeline.predict(X_train)) #finding accuracy score of SGD train
print(f"Accuracy Score: {sgd_train_accuracy:.2f}") #printing accuracy score and other metrics
sgd_train_precision = precision_score(y_train, sgd_pipeline.predict(X_train), average='weighted')
sgd_train_recall = recall_score(y_train, sgd_pipeline.predict(X_train), average='weighted')
sgd_train_f1 = f1_score(y_train, sgd_pipeline.predict(X_train), average='weighted')
print(f"Precision: {sgd_train_precision:.2f}")
print(f"Recall: {sgd_train_recall:.2f}")
print(f"F1 Score: {sgd_train_f1:.2f}")

Accuracy Score: 0.48
Precision: 0.49
Recall: 0.48
F1 Score: 0.47


In [19]:
sgd_test_accuracy = accuracy_score(y_test, sgd_pipeline.predict(X_test)) #finding accuracy score of SGD test
sgd_test_precision = precision_score(y_test, sgd_pipeline.predict(X_test), average='weighted')
sgd_test_recall = recall_score(y_test, sgd_pipeline.predict(X_test), average='weighted')
sgd_test_f1 = f1_score(y_test, sgd_pipeline.predict(X_test), average='weighted')
print(f"Accuracy Score: {sgd_test_accuracy:.2f}") #printing accuracy score and other metrics
print(f"Precision: {sgd_test_precision:.2f}")
print(f"Recall: {sgd_test_recall:.2f}")
print(f"F1 Score: {sgd_test_f1:.2f}") #printing accuracy score

Accuracy Score: 0.45
Precision: 0.46
Recall: 0.45
F1 Score: 0.44


In [20]:
svm_pipeline = make_pipeline(StandardScaler(), LinearSVC(C=1, dual=True, random_state=42))#creating SVM pipeline
svm_pipeline.fit(X_train, y_train)#fitting new model

/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Pipeline(steps=[('standardscaler', StandardScaler()),
                ('linearsvc', LinearSVC(C=1, random_state=42))])

In [21]:
svm_train_accuracy = accuracy_score(y_train, svm_pipeline.predict(X_train)) #finding accuracy score for svm train
svm_train_precision = precision_score(y_train, svm_pipeline.predict(X_train), average='weighted')
svm_train_recall = recall_score(y_train, svm_pipeline.predict(X_train), average='weighted')
svm_train_f1 = f1_score(y_train, svm_pipeline.predict(X_train), average='weighted')
print(f"Accuracy Score: {svm_train_accuracy:.2f}") #printing accuracy score and other metrics
print(f"Precision: {svm_train_precision:.2f}")
print(f"Recall: {svm_train_recall:.2f}")
print(f"F1 Score: {svm_train_f1:.2f}")

Accuracy Score: 0.54
Precision: 0.55
Recall: 0.54
F1 Score: 0.52


In [22]:
svm_test_accuracy = accuracy_score(y_test, svm_pipeline.predict(X_test)) #finding accuracy score of SGD test
svm_test_precision = precision_score(y_test, svm_pipeline.predict(X_test), average='weighted')
svm_test_recall = recall_score(y_test, svm_pipeline.predict(X_test), average='weighted')
svm_test_f1 = f1_score(y_test, svm_pipeline.predict(X_test), average='weighted')
print(f"Accuracy Score: {svm_test_accuracy:.2f}") #printing accuracy score and other metrics
print(f"Precision: {svm_test_precision:.2f}")
print(f"Recall: {svm_test_recall:.2f}")
print(f"F1 Score: {svm_test_f1:.2f}") #printing accuracy score

Accuracy Score: 0.57
Precision: 0.55
Recall: 0.57
F1 Score: 0.54


In [23]:
svm_poly_pipeline = make_pipeline(StandardScaler(), PolynomialFeatures(degree=2),
                                  LinearSVC(C=1, dual=True, random_state=42)) #making polynomial pipeline
svm_poly_pipeline.fit(X_train, y_train)

svm_poly_train_accuracy = accuracy_score(y_train, svm_poly_pipeline.predict(X_train)) #finding accuracy score for svm poly train
svm_poly_train_precision = precision_score(y_train, svm_poly_pipeline.predict(X_train), average='weighted')
svm_poly_train_recall = recall_score(y_train, svm_poly_pipeline.predict(X_train), average='weighted')
svm_poly_train_f1 = f1_score(y_train, svm_poly_pipeline.predict(X_train), average='weighted')
print(f"Accuracy Score: {svm_poly_train_accuracy:.2f}") #printing accuracy score and other metrics
print(f"Precision: {svm_poly_train_precision:.2f}")
print(f"Recall: {svm_poly_train_recall:.2f}")
print(f"F1 Score: {svm_poly_train_f1:.2f}")

Accuracy Score: 0.77
Precision: 0.77
Recall: 0.77
F1 Score: 0.76


/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [24]:
svm_poly_test_accuracy = accuracy_score(y_test, svm_poly_pipeline.predict(X_test))
#finding accuracy for svm poly test

svm_poly_test_accuracy = accuracy_score(y_test, svm_poly_pipeline.predict(X_test)) #finding accuracy score for svm train
svm_poly_test_precision = precision_score(y_test, svm_poly_pipeline.predict(X_test), average='weighted')
svm_poly_test_recall = recall_score(y_test, svm_poly_pipeline.predict(X_test), average='weighted')
svm_poly_test_f1 = f1_score(y_test, svm_poly_pipeline.predict(X_test), average='weighted')
print(f"Accuracy Score: {svm_poly_test_accuracy:.2f}") #printing accuracy score and other metrics
print(f"Precision: {svm_poly_test_precision:.2f}")
print(f"Recall: {svm_poly_test_recall:.2f}")
print(f"F1 Score: {svm_poly_test_f1:.2f}")

Accuracy Score: 0.66
Precision: 0.65
Recall: 0.66
F1 Score: 0.64


# Discussion / Comparison
The SGD models turned out to perform slightly worse than the SVM models on both the train and test set of data. The accuracy, precision, recall, and F1 scores were higher on the training set than the test set for the SGD model. However, the inverse was true for the testing set on the SVM models. All 4 of these models performed worse than the SVC polynomial models. Both of these model's results (training and testing) performed significantly better than the non-polynomial models. Out of the 2, the model performed best on the testing set as the accuracy, precision, recall, and F1 scores were all very close to .77, which is much higher than any other model's results.

#Mastery Level Question

Using your SVM model, produce a boundary decision plot based on the two most important features for predicting the level of obesity.  Discuss/comment on how visually well your model performed compared to the performance statistics.    

#Exercise 2 - Use the AUTOMPG data in order to train an SVM classifier to predict the number of cylinders for an automobile.

After setting up a stratified sampling according to model year, create a pipeline method which incorporates standardscaler, and an SVC classifier to predict the number of cylinders. Compare your model's accuracy to the previous softmax logistic regressor from HW #5 (you should use the solution I've provided).  

Explore adding polynomial features of degree 2 and 3 and compare and discuss all of your models' results.  

For each model, be sure to:
* justify your choice of which features you used for training
* evaluate your model's accuracy on the training data (using cross_val_score(accuracy))
* evaluate your model's accuracy on the test set (using your model to make test predictions, then comparing those test predictions against the actual target test data)
* present your final discussion/comparison in a seprate Markdown cell at the end of your analysis

### Retrieve and preparing auto_mpg data
This series of cells
* retrieves dataset
* replaces a few horsepowers that were missing (looked up on google), change datatype to int
* drops origin and model info



In [25]:
web_path = 'http://public.gettysburg.edu/~rjohnson/ds325/' #if using data over web
auto_df = pd.read_csv(web_path+'auto_mpg.csv')
#auto_df.head()

In [26]:
auto_df.describe()

,mpg,cylinders,displacement,weight,acceleration,model year,origin
count,398.000000,398.000000,398.000000,398.000000,398.000000,398.000000,398.000000
mean,23.514573,5.454774,193.425879,2970.424623,15.568090,76.010050,1.572864
std,7.815984,1.701004,104.269838,846.841774,2.757689,3.697627,0.802055
min,9.000000,3.000000,68.000000,1613.000000,8.000000,70.000000,1.000000
25%,17.500000,4.000000,104.250000,2223.750000,13.825000,73.000000,1.000000
50%,23.000000,4.000000,148.500000,2803.500000,15.500000,76.000000,1.000000
75%,29.000000,8.000000,262.000000,3608.000000,17.175000,79.000000,2.000000
max,46.600000,8.000000,455.000000,5140.000000,24.800000,82.000000,3.000000


In [27]:
auto_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 398 entries, 0 to 397
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   mpg           398 non-null    float64
 1   cylinders     398 non-null    int64  
 2   displacement  398 non-null    float64
 3   horsepower    398 non-null    object 
 4   weight        398 non-null    int64  
 5   acceleration  398 non-null    float64
 6   model year    398 non-null    int64  
 7   origin        398 non-null    int64  
 8   car name      398 non-null    object 
dtypes: float64(3), int64(4), object(2)
memory usage: 28.1+ KB


In [28]:
auto_df[(auto_df['horsepower']=='?')]

,mpg,cylinders,displacement,horsepower,weight,acceleration,model year,origin,car name
32,25.0,4,98.0,?,2046,19.0,71,1,ford pinto
126,21.0,6,200.0,?,2875,17.0,74,1,ford maverick
330,40.9,4,85.0,?,1835,17.3,80,2,renault lecar deluxe
336,23.6,4,140.0,?,2905,14.3,80,1,ford mustang cobra
354,34.5,4,100.0,?,2320,15.8,81,2,renault 18i
374,23.0,4,151.0,?,3035,20.5,82,1,amc concord dl


In [29]:
# what about cylinders 5 and 3?
auto_df[(auto_df['cylinders']==5)]

,mpg,cylinders,displacement,horsepower,weight,acceleration,model year,origin,car name
274,20.3,5,131.0,103,2830,15.9,78,2,audi 5000
297,25.4,5,183.0,77,3530,20.1,79,2,mercedes benz 300d
327,36.4,5,121.0,67,2950,19.9,80,2,audi 5000s (diesel)


In [30]:
# there are 6 ? in the horsepower column, what models are they?
#the 71 ford pinto had 76 hp
#the 74 ford maverick had a 200 in^3 engine with 91 hp
#the 80 renault lecar delux had 55 hp
#the 80 ford mustang cobra had 132 hp
#the 81 renault 18i had 81 hp
#the 82 amc concord dl had 110 hp
# replace those values for horsepower and then convert column to int type

#auto_df['horsepower']=='?'
#auto_df[(auto_df['horsepower']=='?'),1]

#df2 = auto_df
auto_df.replace({'?': ['76','91','55','132','81','110']}, inplace=True)


In [31]:
auto_df['horsepower'][32], auto_df['horsepower'][126], auto_df['horsepower'][330], auto_df['horsepower'][336], auto_df['horsepower'][354], auto_df['horsepower'][374]

('76', '91', '55', '132', '81', '110')

In [32]:
auto_df = auto_df.astype({'horsepower': 'int64'})

In [33]:
auto_df.drop('origin', axis=1, inplace=True)
auto_df.drop('car name', axis=1, inplace=True)

In [34]:
auto_df.info()
#auto_df['horsepower'][1]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 398 entries, 0 to 397
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   mpg           398 non-null    float64
 1   cylinders     398 non-null    int64  
 2   displacement  398 non-null    float64
 3   horsepower    398 non-null    int64  
 4   weight        398 non-null    int64  
 5   acceleration  398 non-null    float64
 6   model year    398 non-null    int64  
dtypes: float64(3), int64(4)
memory usage: 21.9 KB


# Solution

In [35]:
X = auto_df.drop('cylinders', axis=1, inplace=False) #defining X and y
y = auto_df['cylinders']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, #train test split
                                                    stratify = auto_df['model year'])
# and making sure to stratify model year variable as well

svm_pipeline = make_pipeline(StandardScaler(), LinearSVC(C=1, dual=True, random_state=42))
svm_pipeline.fit(X_train, y_train) #creating pipeline

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('linearsvc', LinearSVC(C=1, random_state=42))])

In [36]:
svm_train_accuracy = accuracy_score(y_train, svm_pipeline.predict(X_train)) #finding metrics for svm train
svm_train_precision = precision_score(y_train, svm_pipeline.predict(X_train), average='weighted')
svm_train_recall = recall_score(y_train, svm_pipeline.predict(X_train), average='weighted')
svm_train_f1 = f1_score(y_train, svm_pipeline.predict(X_train), average='weighted')
print(f"Accuracy Score: {svm_train_accuracy:.2f}") #printing accuracy score and other metrics
print(f"Precision: {svm_train_precision:.2f}")
print(f"Recall: {svm_train_recall:.2f}")
print(f"F1 Score: {svm_train_f1:.2f}")

Accuracy Score: 0.94
Precision: 0.92
Recall: 0.94
F1 Score: 0.93


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [37]:
svm_cv_scores = cross_val_score(svm_pipeline, X_train, y_train, cv=10, scoring='accuracy')#cross validation
print("Cross-validation scores:", svm_cv_scores)# Print the cross-validation scores

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=10.
  warnings.warn(


Cross-validation scores: [0.9375     1.         0.90625    0.875      0.90625    0.96875
 0.9375     0.84375    0.96774194 0.96774194]


In [38]:
svm_test_accuracy = accuracy_score(y_test, svm_pipeline.predict(X_test)) #finding metrics for svm test
svm_test_precision = precision_score(y_test, svm_pipeline.predict(X_test), average='weighted')
svm_test_recall = recall_score(y_test, svm_pipeline.predict(X_test), average='weighted')
svm_test_f1 = f1_score(y_test, svm_pipeline.predict(X_test), average='weighted')
print(f"Accuracy Score: {svm_test_accuracy:.2f}") #printing accuracy score and other metrics
print(f"Precision: {svm_test_precision:.2f}")
print(f"Recall: {svm_test_recall:.2f}")
print(f"F1 Score: {svm_test_f1:.2f}")

Accuracy Score: 0.93
Precision: 0.91
Recall: 0.93
F1 Score: 0.91


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [39]:
svm_poly_pipeline = make_pipeline(StandardScaler(), PolynomialFeatures(degree=2),
                                  LinearSVC(C=1, dual=True, random_state=42)) #making polynomial pipeline
svm_poly_pipeline.fit(X_train, y_train)

svm_poly_train_accuracy = accuracy_score(y_train, svm_poly_pipeline.predict(X_train)) #finding accuracy score for svm train
svm_poly_train_precision = precision_score(y_train, svm_poly_pipeline.predict(X_train), average='weighted')
svm_poly_train_recall = recall_score(y_train, svm_poly_pipeline.predict(X_train), average='weighted')
svm_poly_train_f1 = f1_score(y_train, svm_poly_pipeline.predict(X_train), average='weighted')
print(f"Accuracy Score: {svm_poly_train_accuracy:.2f}") #printing accuracy score and other metrics
print(f"Precision: {svm_poly_train_precision:.2f}")
print(f"Recall: {svm_poly_train_recall:.2f}")
print(f"F1 Score: {svm_poly_train_f1:.2f}")

/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Accuracy Score: 0.99
Precision: 0.99
Recall: 0.99
F1 Score: 0.99


In [40]:
svm_poly_pipeline = make_pipeline(StandardScaler(), PolynomialFeatures(degree=3),
                                  LinearSVC(C=1, dual=True, random_state=42)) #making polynomial pipeline
svm_poly_pipeline.fit(X_train, y_train)

svm_poly_train_accuracy = accuracy_score(y_train, svm_poly_pipeline.predict(X_train)) #finding accuracy score for svm train
svm_poly_train_precision = precision_score(y_train, svm_poly_pipeline.predict(X_train), average='weighted')
svm_poly_train_recall = recall_score(y_train, svm_poly_pipeline.predict(X_train), average='weighted')
svm_poly_train_f1 = f1_score(y_train, svm_poly_pipeline.predict(X_train), average='weighted')
print(f"Accuracy Score: {svm_poly_train_accuracy:.2f}") #printing accuracy score and other metrics
print(f"Precision: {svm_poly_train_precision:.2f}")
print(f"Recall: {svm_poly_train_recall:.2f}")
print(f"F1 Score: {svm_poly_train_f1:.2f}")

Accuracy Score: 0.99
Precision: 0.99
Recall: 0.99
F1 Score: 0.99


/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


# Discussion / Conclusions
For these particular models, I chose to use all features (outside of the target feature) so that the results would be fully comparable to the results from HW 5. The SVM model appears to have performed slightly worse on the train and test sets than the model from HW 5, but the margin is slim. The accuracy scores on the testing and training sets for the results below were .94 and .93 respecively. The scores on HW 5 were .95 and .93 respectively. These results are very close but the HW 5 model does indeed perform better on many of the other metrics as well. However, when adding in the polynomial features, the models performed significantly better than the previous two, and the models in HW 5. Both models produced results with all metrics close to .99, implying that these are the best performing models.

# Mastery Level Question
For the previous problem, optimize your model hyperparameters **C** (the regularization parameter) and **power** (from PolynomialFeatures) and present the optimal values and model results for each.

# Exercise 3 - Comparing SGD to SVM classifier on MNIST dataset

Use PCA to reduce the size of the MNIST dataset, retaining 95% of the variance.  Then, train an SVM classifier on the dataset and evaluate it against training and test data.

Use PCA again to reduce the size of the MNIST dataset, this time retaining only 75% of the variance.  Then, train an SVM classifier on the dataset, adding in PolynomialFeatures of degree 2 and evaluate it against training and test data.

Quantitatively compare and discuss your models' performances to eachother, and to the SGD model that was used for this same purpose in HW #5 (you may use the solution I've provided).   


# Solution

In [41]:
from keras.datasets import mnist #importing necessary materials
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score
(X_train, y_train), (X_test, y_test) = mnist.load_data() #loading in new data set with X and y train / test
X_train_resized = np.resize(X_train, (60000, 784)) #resizing X train and X test
X_test_resized = np.resize(X_test, (10000, 784))
pca = PCA(n_components=0.95) #specifying variance for transformation
X_train_reduced = pca.fit_transform(X_train_resized)#transforming X_train reduced
X_test_reduced = pca.transform(X_test_resized)
svm_classifier_95 = SVC()
svm_classifier_95.fit(X_train_reduced, y_train)

SVC()

In [42]:
y_train_pred = svm_classifier_95.predict(X_train_reduced) #predicting on training and test sets
y_test_pred = svm_classifier_95.predict(X_test_reduced)

In [43]:
train_accuracy = accuracy_score(y_train, y_train_pred) # Calculating metrics for train data
train_precision = precision_score(y_train, y_train_pred, average='weighted')
train_recall = recall_score(y_train, y_train_pred, average='weighted')
train_f1 = f1_score(y_train, y_train_pred, average='weighted')

test_accuracy = accuracy_score(y_test, y_test_pred) # Calculating metrics for test data
test_precision = precision_score(y_test, y_test_pred, average='weighted')
test_recall = recall_score(y_test, y_test_pred, average='weighted')
test_f1 = f1_score(y_test, y_test_pred, average='weighted')

In [44]:
print("Evaluation Metrics for Training Data:") #printing metrics for test and training sets
print(f"Accuracy: {train_accuracy:.4f}")
print(f"Precision: {train_precision:.4f}")
print(f"Recall: {train_recall:.4f}")
print(f"F1-score: {train_f1:.4f}\n")

print("Evaluation Metrics for Test Data:")
print(f"Accuracy: {test_accuracy:.4f}")
print(f"Precision: {test_precision:.4f}")
print(f"Recall: {test_recall:.4f}")
print(f"F1-score: {test_f1:.4f}")

Evaluation Metrics for Training Data:
Accuracy: 0.9934
Precision: 0.9934
Recall: 0.9934
F1-score: 0.9934

Evaluation Metrics for Test Data:
Accuracy: 0.9832
Precision: 0.9832
Recall: 0.9832
F1-score: 0.9832


In [45]:
SGD_classifier_75 = SGDClassifier(max_iter=10000, random_state=42) #defining SGD classifier
SGD_classifier_75.fit(X_train_reduced, y_train)

SGDClassifier(max_iter=10000, random_state=42)

In [46]:
pca_75 = PCA(n_components=0.75) #defining new PCA
X_train_reduced_75 = pca_75.fit_transform(X_train_resized)
X_test_reduced_75 = pca_75.transform(X_test_resized)

svm_poly_classifier_75 = make_pipeline(PCA(n_components=0.75), PolynomialFeatures(degree=2), SVC())
svm_poly_classifier_75.fit(X_train_resized, y_train) #creating and fitting new pipeline model

Pipeline(steps=[('pca', PCA(n_components=0.75)),
                ('polynomialfeatures', PolynomialFeatures()), ('svc', SVC())])

In [48]:
y_train_pred_75 = svm_poly_classifier_75.predict(X_train_resized) #predictions on training set
#y_test_pred_75 = svm_poly_classifier_75.predict(X_test_reduced_75)

In [49]:
train_accuracy_75 = accuracy_score(y_train, y_train_pred_75) #finding metrics
train_precision_75 = precision_score(y_train, y_train_pred_75, average='weighted')
train_recall_75 = recall_score(y_train, y_train_pred_75, average='weighted')
train_f1_75 = f1_score(y_train, y_train_pred_75, average='weighted')

In [50]:
print("Evaluation Metrics for Training Data (75% variance retained):") #printing metrics
print(f"Accuracy: {train_accuracy_75:.4f}")
print(f"Precision: {train_precision_75:.4f}")
print(f"Recall: {train_recall_75:.4f}")
print(f"F1-score: {train_f1_75:.4f}\n")

Evaluation Metrics for Training Data (75% variance retained):
Accuracy: 0.9928
Precision: 0.9928
Recall: 0.9928
F1-score: 0.9927



# Discussion / Conclusions
The dataset that retains 95% of the variance produces good restuls overall; however, the training data set performs slighly better than model on the testing set. The metrics on the training set are all close to .99 but the metrics on the testing set are closer to .98. The model that retain 75% of the variance performs well overall with most of the metrics close to .99 on the testing set. This is very similar to the metrics produced by the data set that retains 95% of the variance. Compared to the results in HW 5, both models here appear to perform significantly better as the metrics are overall closer to 1 than the metrics in the HW 5. Note: I attempted to create predictions on the test set for the model that retained 75% of the variance, but the code was not able to run on my laptop so I commented the code out.